## **Algoritmo de filtraje no lineal basado en Operador de Koopman aplicado a epidemiología**

### **Capítulo 3: Kernel Extended Dynamic Mode Decomposition**

**Autor: Diego Olguín.**

**Supervisores: Héctor Ramírez y Axel Osses.**

In [1]:
chapter = "chapter3/"
img_path = "img/content/"+chapter

In [2]:
# Librerías generales
import numpy as np
from scipy import stats
from sklearn.gaussian_process.kernels import Matern

# Plotly
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Librerías propias
import os

# Dependencias de KKKF
from KKKF.DynamicalSystems import DynamicalSystem
from KKKF.kEDMD import KoopmanOperator

In [3]:
# Ajustar el tamaño de las figuras y los títulos para Plotly
pio.templates.default = 'plotly'
pio.templates['plotly'].layout.update(
    title={'font': {'size': 24}},
    xaxis={'title': {'font': {'size': 20}}},
    yaxis={'title': {'font': {'size': 20}}},
    legend={'font': {'size': 16}},
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
);

In [4]:
# Ajustar directorio para las imagénes
root = "/Users/diegoolguin/Koopman_nonlinear_filter"
if not os.getcwd()==root:
    os.chdir(root)

### **kEDMD en el caso de modelos lineales**

In [5]:
# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 2500
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=3*np.eye(3))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Datos sintéticos
iters = 50
x0 = np.array([0.1, 0.1, 0.1])

# Tiempo
t = np.arange(iters)

In [6]:
# Función auxiliar para cálculo de kEDMD en el caso lineal
def compute_linear_kedmd(alpha, N):
    # Matriz
    A = np.array([
        [0.01, 0.04, 0.0],
        [0.01, 0.02, alpha],
        [0.0, 0.04, 0.02]
    ])

    # Función de dinámica
    f = lambda x: x + A@x

    # Solución real
    x = np.zeros((iters, nx))
    x[0] = x0

    for i in range(1, iters):
        x[i] = f(x[i-1])

    # Sistema dinámico
    dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

    # Operador de Koopman
    Koop = KoopmanOperator(k, dyn)

    # Se calcula kEDMD
    Koop.compute_edmd(N)
    U, B, phi = Koop.U, Koop.B, Koop.phi

    # Sistema con Koopman
    x_koop = np.zeros((iters, nx))
    x_koop[0] = B@phi(x0)
    z = np.zeros((iters, N))
    z[0] = phi(x0)

    for i in range(1, iters):
        z[i] = U@z[i-1]
        x_koop[i] = B@z[i]

    return x, x_koop, A, U

In [7]:
# Alpha
alpha = -0.3

# Predicción
x, x_koop, A, U = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='x1 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='x2 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='x3 real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='x1 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='x2 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='x3 predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"Linear1.pdf")
fig.show()

In [8]:
# Alpha
alpha = -0.1

# Predicción
x, x_koop, A, U = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='x1 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='x2 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='x3 real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='x1 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='x2 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='x3 predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"Linear2.pdf")
fig.show()

In [9]:
# Alpha
alpha = 0.05

# Predicción
x, x_koop, A, U = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='x1 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='x2 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='x3 real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='x1 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='x2 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='x3 predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"Linear3.pdf")
fig.show()

In [10]:
# Alpha a utilizar
alpha = -0.3

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop, _, _ = compute_linear_kedmd(alpha, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo lineal con matriz de parámetro {alpha}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"Linear1Errors.pdf")

fig.show()

In [11]:
# Alpha a utilizar
alpha = -0.1

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop, _, _  = compute_linear_kedmd(alpha, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo lineal con matriz de parámetro {alpha}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"Linear2Errors.pdf")

fig.show()

In [12]:
# Alpha a utilizar
alpha = 0.05

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop, _, _  = compute_linear_kedmd(alpha, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo lineal con matriz de parámetro {alpha}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"Linear3Errors.pdf")

fig.show()

### **Visualizando la distribución Dirichlet**

In [13]:
# Distintos casos de distribución Dirichlet
N_samples = 1000

# Distribuciones
dirichlet1 = stats.dirichlet(alpha=[1, 1, 1])
dirichlet2 = stats.dirichlet(alpha=[7, 7, 7])
dirichlet3 = stats.dirichlet(alpha=[1, 5, 10])

# Muestras
samples1 = dirichlet1.rvs(N_samples)
samples2 = dirichlet2.rvs(N_samples)
samples3 = dirichlet3.rvs(N_samples)

# Gráfico en 3D
fig = make_subplots(
    rows=1, 
    cols=3, 
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=['Dirichlet (1, 1, 1)', 'Dirichlet (7, 7, 7)', 'Dirichlet (1, 5, 10)']
)
# Gráfico 1
fig.add_trace(go.Scatter3d(
    x=samples1[:,0], y=samples1[:,1], z=samples1[:,2],
    mode='markers', marker=dict(size=3), name='Dirichlet (1, 1, 1)'
), row=1, col=1)

# Gráfico 2
fig.add_trace(go.Scatter3d(
    x=samples2[:,0], y=samples2[:,1], z=samples2[:,2],
    mode='markers', marker=dict(size=3), name='Dirichlet (7, 7, 7)'
), row=1, col=2)

# Gráfico 3
fig.add_trace(go.Scatter3d(
    x=samples3[:,0], y=samples3[:,1], z=samples3[:,2],
    mode='markers', marker=dict(size=3), name='Dirichlet (1, 5, 10)'
), row=1, col=3)

# Colocar titulo
fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    height=400,
    width=1400
    )

# Límites de los ejes
fig.update_scenes(
    xaxis = dict(range=[0, 1]),
    yaxis = dict(range=[0, 1]),
    zaxis = dict(range=[0, 1])
)

# No mostrar la leyenda
fig.update_layout(showlegend=False)

fig.write_image(img_path+"Dirichlet.pdf")
fig.show()

### **kEDMD para modelo SIR**

In [14]:
# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 1000
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.dirichlet(alpha=1*np.ones(nx))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(nx))
obs_dist = stats.multivariate_normal(mean=np.zeros(ny), cov=1e-7*np.eye(ny))


# Datos sintéticos
iters = 20
x0 = np.array([0.9, 0.1, 0.0])

In [15]:
# Función para calcular kEDMD en el modelo SIR
def kEDMD_SIR(beta, gamma, N):
    # Función de dinámica
    f = lambda x: x + np.array([
        -beta*x[0]*x[1],
        beta*x[0]*x[1] - gamma*x[1],
        gamma*x[1]
    ])

    # Solución real
    x = np.zeros((iters, nx))
    x[0] = x0

    for i in range(1, iters):
        x[i] = f(x[i-1])

    # Sistema dinámico
    dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

    # Operador de Koopman
    Koop = KoopmanOperator(k, dyn)

    # Se calcula kEDMD
    Koop.compute_edmd(N)
    U, B, phi = Koop.U, Koop.B, Koop.phi

    # Sistema con Koopman
    x_koop = np.zeros((iters, nx))
    x_koop[0] = B@phi(x0)
    z = np.zeros((iters, N))
    z[0] = phi(x0)

    for i in range(1, iters):
        z[i] = U@z[i-1]
        x_koop[i] = B@z[i]

    return x, x_koop

In [16]:
# Parámetros
beta1, gamma1 = 1, 0.3
beta2, gamma2 = 0.5, 0.1
beta3, gamma3 = 1.5, 0.6

# Predicción
x1, x_koop1 = kEDMD_SIR(beta1, gamma1, N)
x2, x_koop2 = kEDMD_SIR(beta2, gamma2, N)
x3, x_koop3 = kEDMD_SIR(beta3, gamma3, N)

In [17]:
# Gráfico para los primeros parámetros
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(iters), y=x1[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x1[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x1[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop1[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop1[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop1[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo SIR con beta={beta1}, gamma={gamma1}",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"SIR1.pdf")
fig.show()

In [18]:
# Gráfico para los segundos parámetros
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(iters), y=x2[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x2[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x2[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop2[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop2[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop2[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo SIR con beta={beta2}, gamma={gamma2}",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"SIR2.pdf")
fig.show()

In [19]:
# Gráfico para los terceros parámetros
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(iters), y=x3[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x3[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x3[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop3[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop3[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop3[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo SIR con beta={beta3}, gamma={gamma3}",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada",
    margin=dict(l=0, r=0, t=50, b=0)
    )

fig.write_image(img_path+"SIR3.pdf")
fig.show()

In [20]:
# Parámetros a utilizar
beta, gamma = beta1, gamma1

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = kEDMD_SIR(beta, gamma, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo SIR de parámetros beta = {beta}, gamma = {gamma}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR1Errors.pdf")

fig.show()

In [21]:
# Parámetros a utilizar
beta, gamma = beta2, gamma2

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = kEDMD_SIR(beta, gamma, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo SIR de parámetros beta = {beta}, gamma = {gamma}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR2Errors.pdf")

fig.show()

In [22]:
# Parámetros a utilizar
beta, gamma = beta3, gamma3

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = kEDMD_SIR(beta, gamma, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo SIR de parámetros beta = {beta}, gamma = {gamma}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR3Errors.pdf")

fig.show()

### **kEDMD para modelo SIR con recuperación**

In [23]:
# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 1000
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.dirichlet(alpha=1*np.ones(nx))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(nx))
obs_dist = stats.multivariate_normal(mean=np.zeros(ny), cov=1e-7*np.eye(ny))


# Datos sintéticos
iters = 50
x0 = np.array([0.9, 0.1, 0.0])

In [24]:
# Función para calcular kEDMD en el modelo SIR con recuperación
def kEDMD_SIR_rec(alpha, beta, gamma, N):
    # Función de dinámica
    f = lambda x: x + np.array([
        -beta*x[0]*x[1] + alpha*x[2],
        beta*x[0]*x[1] - gamma*x[1],
        gamma*x[1] - alpha*x[2]
    ])

    # Solución real
    x = np.zeros((iters, nx))
    x[0] = x0

    for i in range(1, iters):
        x[i] = f(x[i-1])

    # Sistema dinámico
    dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

    # Operador de Koopman
    Koop = KoopmanOperator(k, dyn)

    # Se calcula kEDMD
    Koop.compute_edmd(N)
    U, B, phi = Koop.U, Koop.B, Koop.phi

    # Sistema con Koopman
    x_koop = np.zeros((iters, nx))
    x_koop[0] = B@phi(x0)
    z = np.zeros((iters, N))
    z[0] = phi(x0)

    for i in range(1, iters):
        z[i] = U@z[i-1]
        x_koop[i] = B@z[i]

    return x, x_koop

In [25]:
# Parámetros
alpha1, beta1, gamma1 = 0.1, 1, 0.3
alpha2, beta2, gamma2 = 0.3, 0.5, 0.1
alpha3, beta3, gamma3 = 0.5, 1.5, 0.6

# Predicción
x1, x_koop1 = kEDMD_SIR_rec(alpha1, beta1, gamma1, N)
x2, x_koop2 = kEDMD_SIR_rec(alpha2, beta2, gamma2, N)
x3, x_koop3 = kEDMD_SIR_rec(alpha3, beta3, gamma3, N)

In [26]:
# Gráfico para los primeros parámetros
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(iters), y=x1[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x1[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x1[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop1[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop1[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop1[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo con recuperación, alpha={alpha1}, beta={beta1}, gamma={gamma1}",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR_rec1.pdf")
fig.show()

In [27]:
# Gráfico para los segundos parámetros
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(iters), y=x2[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x2[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x2[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop2[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop2[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop2[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo con recuperación, alpha={alpha2}, beta={beta2}, gamma={gamma2}",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR_rec2.pdf")
fig.show()

In [28]:
# Gráfico para los terceros parámetros
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(iters), y=x3[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x3[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x3[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop3[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop3[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_koop3[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo con recuperación, alpha={alpha3}, beta={beta3}, gamma={gamma3}",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR_rec3.pdf")
fig.show()

In [29]:
# Parámetros a utilizar
alpha, beta, gamma = alpha1, beta1, gamma1

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = kEDMD_SIR_rec(alpha, beta, gamma, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Modelo con recuperación de parámetros alpha={alpha}, beta = {beta}, gamma = {gamma}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR_rec1Errors.pdf")

fig.show()

In [30]:
# Parámetros a utilizar
alpha, beta, gamma = alpha2, beta2, gamma2

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = kEDMD_SIR_rec(alpha, beta, gamma, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Modelo con recuperación de parámetros alpha={alpha}, beta = {beta}, gamma = {gamma}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR_rec2Errors.pdf")

fig.show()

In [31]:
# Parámetros a utilizar
alpha, beta, gamma = alpha3, beta3, gamma3

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = kEDMD_SIR_rec(alpha, beta, gamma, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Modelo con recuperación de parámetros alpha={alpha}, beta = {beta}, gamma = {gamma}",
    xaxis_title="Número de puntos",
    yaxis_title="Error",
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
    )

fig.write_image(img_path+"SIR_rec3Errors.pdf")

fig.show()